# Regression and Model Fitting

K. Leighly 2017

This lecture was drawn from the following sources:
 - Ivezic chapter 8
 - Bishop Chapter 12
 - Numerical Recipes
 - Kelly (2007) "Some Aspects of Measurement Error in Linear Regression of Astronomical Data", ApJ 665, 1489


## Introduction and Motivation

In this lecture we consider regression, a special case of general model fitting.

We have already touched on this topic in lecture?, when we talked about $\chi^2$ and least-squares fitting.  Here, we will talk about model fitting more generally.  But let's return to that example in order to see what can result from out-of-control model fitting.


### Outine
- Motivation 
- Basic Development of Linear Regression Formalism
- Regularization
- Overfitting, Underfitting, and Crossvalidation
- Including Measurement Error and Limits.

In [ ]:
## Initiate python stuff
%pylab inline
import scipy.stats

In [ ]:
# Set up an x-vector
x=np.linspace(0,1,11)

# Set up an x-vector with the same range, but sampled more finely
x2=np.linspace(0,1,101)

# y is a sine wave
y=np.sin(2*np.pi*x)

#give y some uncertainty, i.e., apply a Gaussian scatter around 
#the theoretically determined values.
ynew=y+np.random.normal(0,0.15,11)

#assign some errors.  These errors don't have any physical interpretation, 
#but are reasonably appropriate, as they are scattered around the 
#sigma of the uncertainty gaussian (=0.15)
err=np.random.uniform(0.1,0.2,11)


#plot ideal y, and data.
plt.plot(x,y)
plt.errorbar(x, ynew, yerr=err, fmt='o')

In [ ]:
#fit the data with a zeroth order polynomial using polyfit
result0=np.polyfit(x,ynew,0,w=1/err)
print 'The best fit coefficients ',result0

#poly1d will generate the model using the best fit coefficients.  
y0=np.poly1d(result0)
y0out=y0(x)
plt.errorbar(x, ynew, yerr=err, fmt='o')
plt.plot(x,y0out)

As before, we can do polynomial fits of increasing order.  
 - np.polyfit yields the polynomial coefficients
 - np.poly1d is used to predict the y values for those coefficients. 

In [ ]:
result1=np.polyfit(x,ynew,1,w=1/err)
result2=np.polyfit(x,ynew,2,w=1/err)
y2=np.poly1d(result2)
y2out2=y2(x2)
y2out=y2(x)

result3=np.polyfit(x,ynew,3,w=1/err)
y3=np.poly1d(result3)
y3out2=y3(x2)
y3out=y3(x)

result4=np.polyfit(x,ynew,4,w=1/err)
y4=np.poly1d(result4)
y4out2=y4(x2)
y4out=y4(x)

result5=np.polyfit(x,ynew,5,w=1/err)
y5=np.poly1d(result5)
y5out2=y5(x2)
y5out=y5(x)

result6=np.polyfit(x,ynew,6,w=1/err)
y6=np.poly1d(result6)
y6out2=y6(x2)
y6out=y6(x)

result7=np.polyfit(x,ynew,7,w=1/err)
y7=np.poly1d(result7)
y7out2=y7(x2)
y7out=y7(x)

result8=np.polyfit(x,ynew,8,w=1/err)
y8=np.poly1d(result8)
y8out2=y8(x2)
y8out=y8(x)

result9=np.polyfit(x,ynew,9,w=1/err)
y9=np.poly1d(result9)
y9out2=y9(x2)
y9out=y9(x)

result10=np.polyfit(x,ynew,10,w=1/err)
y10=np.poly1d(result10)
y10out2=y10(x2)
y10out=y10(x)

In [ ]:
pylab.rcParams['figure.figsize'] = (15, 15)

plt.subplot(3,3,1)
plt.errorbar(x, ynew, yerr=err, fmt='o')
plt.plot(x,y2out)
plt.plot(x2,y2out2)

plt.subplot(3,3,2)
plt.errorbar(x, ynew, yerr=err, fmt='o')
plt.plot(x,y3out)
plt.plot(x2,y3out2)

plt.subplot(3,3,3)
plt.errorbar(x, ynew, yerr=err, fmt='o')
plt.plot(x,y4out)
plt.plot(x2,y4out2)

plt.subplot(3,3,4)
plt.errorbar(x, ynew, yerr=err, fmt='o')
plt.plot(x,y5out)
plt.plot(x2,y5out2)

plt.subplot(3,3,5)
plt.errorbar(x, ynew, yerr=err, fmt='o')
plt.plot(x,y6out)
plt.plot(x2,y6out2)

plt.subplot(3,3,6)
plt.errorbar(x, ynew, yerr=err, fmt='o')
plt.plot(x,y7out)
plt.plot(x2,y7out2)

plt.subplot(3,3,7)
plt.errorbar(x, ynew, yerr=err, fmt='o')
plt.plot(x,y8out)
plt.plot(x2,y8out2)

plt.subplot(3,3,8)
plt.errorbar(x, ynew, yerr=err, fmt='o')
plt.plot(x,y9out)
plt.plot(x2,y9out2)

plt.subplot(3,3,9)
plt.errorbar(x, ynew, yerr=err, fmt='o')
plt.plot(x,y10out)
plt.plot(x2,y10out2)

The above plots show that models of increasing complexity do an increasingly better job of fitting the training set (the original 11 points), but start to do worse and worse at predicting other values of $x$.  This is bad because usually, when you fit a model to some training data,  you have in mind the idea to predict the $y$ values for a new test set $x_{new}$.

Another thing we can look at is the coefficients of the models.  The result of polyfit gives the fit coefficients in decreasing order of power (i.e., the constant offset is the last coefficient in the result vector).  So let's look at the first coefficients for all of the models:

In [ ]:
pylab.rcParams['figure.figsize'] = (6,6)

temp=[result0[0],result1[0],result2[0],result3[0],result4[0],result5[0],result6[0],result7[0],result8[0],result9[0],result10[0]]
plt.plot(temp)

print temp


You can see that the absolute values of these coeffients increase dramatically as the order increase.  In addition, the values of all coefficients in the higher-order fits become large.  Let us compare the result4 and result10 below.

In [ ]:
print result4
print result10

There is another issue that can be illustrated as follows.

$ynew$ has error bars, so if we assume they have a normal distribution, then we can assume there is a reasonable chance that we could have measured $ynew_2$, which is $ynew$ plus a normally distributed random variable with location of zero and width equal to the error bar.  What happens when we subject such data to the same analysis?

In [ ]:
ynew_2=np.zeros(11)
for i in range(11):
    ynew_2[i]=ynew[i]+np.random.normal(0,err[i])
    
plt.errorbar(x, ynew, yerr=err, fmt='o')
plt.errorbar(x, ynew_2, yerr=err,fmt='o')

In [ ]:
result2_0=np.polyfit(x,ynew_2,0,w=1/err)
result2_1=np.polyfit(x,ynew_2,1,w=1/err)
result2_2=np.polyfit(x,ynew_2,2,w=1/err)
result2_3=np.polyfit(x,ynew_2,3,w=1/err)
result2_4=np.polyfit(x,ynew_2,4,w=1/err)
result2_5=np.polyfit(x,ynew_2,5,w=1/err)
result2_6=np.polyfit(x,ynew_2,6,w=1/err)
result2_7=np.polyfit(x,ynew_2,7,w=1/err)
result2_8=np.polyfit(x,ynew_2,8,w=1/err)
result2_9=np.polyfit(x,ynew_2,9,w=1/err)
result2_10=np.polyfit(x,ynew_2,10,w=1/err)

temp_2=[result2_0[0],result2_1[0],result2_2[0],result2_3[0],result2_4[0],result2_5[0],result2_6[0],result2_7[0],result2_8[0],result2_9[0],result2_10[0]]


In [ ]:
pylab.rcParams['figure.figsize'] = (15,6)

plt.subplot(1,2,1)

plt.plot(temp)
plt.plot(temp_2)

temp=np.asarray(temp)

plt.subplot(1,2,2)
plt.plot(temp-temp_2)



We see that the higher order coefficients deviate markedly for the $ynew_2$ model.  

Depending on your purpose for fitting the data, you may have been inclined to report the fit coefficients in a paper (for other people to use) or use the model to predict new data.  Clearly both of those results would have had serious problems in these over-fit situations.

## The Bottom Line

Regression is very commonly done in analyzing data.  It is perhaps one of the most common tasks: fit model (either physical or empirical) to data.  However, as we have seen above, there are potential pitfalls.  Fortunately, there are tools that we can use both to understand our models and to control their behavior.

## Linear Models for Regression

We will now go through the standard development of linear models for regression. This discussion will follow Bishop Chapter 3, but a very similar development is in Ivezic Chapter 8. Another interesting resource (with exercises) is [Hogg et al. 2010](https://arxiv.org/abs/1008.4686).

A first thing to emphasize is that when we perform regression, we need to think about the physics of the situation, because it seems that the approach and desired outcomes may depend on that intimately. 

On the one hand, there is model fitting. For example, consider an X-ray observation of a radio loud quasar, where I am reasonably confident that the intrinsic spectrum is a power law, and I want to find the slope and normalization of that power law, I can use some model fitting software and $\chi^2$ statistics to find the best fitting parameters. This is straightforward because the uncertainty in the data points is statistical, e.g., due to Poission statistics. This becomes somewhat less straightforward when the signal to noise is poor and uncertainties are not Gaussian, but it is still managable (at least, as far as I have found, although you could imagine more complicated scenarios, e.g., calibration uncertainty that you want to propagate through).

The problem that we are discussing in this chapter is somewhat different. In this problem, there is statistical uncertainy, but there is also intrinsic scatter. As discussed by Hogg et al. 2010, it is not always clear that fitting, e.g., a straight line to such data is meaningful, because it is not clear why nature would choose a linear model.  That said, regression can be useful for deriving and comparing scaling relations, for example.  At the same time, one might argue that the values of the fit parameters are rather less important than the derived probability distribution, i.e., as Hogg et al. 2010 argues, deriving a generative model (i.e., one that can predict the $y$ values for new values of $x$) is critical.



A second thing to consider is what is meant by "linear".  We do not mean that you are fitting a straight line, but rather that the fit coefficients are linear.  So, a polynomial is linear, whereas 

$$y(x)=A \exp(-B x)$$

is not linear in the coefficients $A$ and $B$.

However, there is a transformation that will make this a linear equation. (What is that transformation?)

Generally speaking, it is easier to deal with linear models than nonlinear ones, although there is software that will fit a nonlinear model for you.

### Linear Basis Function Models

Again, we are talking about linear regression, i.e., the target is a linear function of the fit parameters. A general way to write this might be:

$$y(\mathbf{x},\mathbf{w}) = \sum_{j=0}^{M-1} w_j \phi_i(\mathbf{x}) = \mathbf{w}^T \boldsymbol{\phi}({\mathbf{x}}) $$

where $\mathbf{w} = (w_0,\ldots,w_{M-1})^T$ are the parameters and $\boldsymbol{\phi}=(\phi_0,\ldots,\phi_{M-1})^T$ are the basis functions. In this case, $\phi_0 $ is constrained to be equal to one to take care of the bias, i.e., $w_0$.

There are many possible forms for the basis functions.



There are many possible forms for the basis functions, including: 


#### Polynomial and spline basis functions:

We have already discussed the application of polynomial basis functions, $\phi_j (x)=x^j$. 

A disadvantage of polynomial basis functions is the fact that they are global, so that changes in one region of space affect all the other regions. A way around that are spline functions, in which different regions are fit with different polynomials. An example is the cubic spline, where piece-wise order-3 polynomials are used.

Polynomial basis functions include the regular ones, but also ones that are orthogonal, such as the Hermite polynomials.

![Wikipedia](https://upload.wikimedia.org/wikipedia/commons/e/ec/Hermite_poly.svg)





#### Gaussian basis functions:

$$\phi_j(x) = \exp \left \{ -\frac{(x-\mu_j)^2}{2s^2} \right \},$$

where the $\mu_j$ are the locations of the basis functions and $s$ governs their width.

![wikipedia commons](https://upload.wikimedia.org/wikipedia/commons/7/74/Normal_Distribution_PDF.svg)

#### Sigmoidal basis function

$$\phi_j(x) = \sigma \left ( \frac{x-\mu_j}{s} \right ),$$

with $\sigma(a)$ being the logistic sigmoid function defined by

$$\sigma(a)=\frac{1}{1+\exp(-a)}.$$



![wikipedia](https://upload.wikimedia.org/wikipedia/commons/8/88/Logistic-curve.svg)



#### Fourier Basis

The basis functions can be sinusoidal; this seems equivalent to a Fourier decomposition. Again, it may be problematic that these have infinite spatial extent, in which case wavelets could be used as basis functions.

#### Principal components

A truncated set of the principal components derived from principal components analysis may make a suitable basis set.



## Maximum Likelihood and Least Squares

This is the formal derivation of linear regression that is seen almost everywhere.  What this will yield is the formal solution for the coefficients of the basis functions that fit your data.

Assume that the target variable $t$ (the data) is given by a deterministic function $y(\mathbf{x},\mathbf{w})$ (the intrinsic $y$) with additive Gaussian noise so that

$$t=y(\mathbf{x},\mathbf{w}) + \epsilon,$$

where $\epsilon$ is a zero-mean Gaussian random variable with precision (inverse variance) equal to $\beta$. So

$$p(t|\mathbf{x},\mathbf{w},\beta) = \mathcal{N}(t |y(\mathbf{x},\mathbf{w}),\beta^{-1}).$$

We have discussed how, for a squared loss function (i.e., like $\chi^2$), the optimal prediction for a new value of $\mathbf{x}$, will be given by the conditional mean (i.e., the mean weighted by $p$) of the target variable. For the Gaussian distribution above, the conditional mean is:

$$\mathit{E}[t|\mathbf{x}] = \int t p(t|\mathbf{x}) dt = y(\mathbf{x},\mathbf{w}). $$

This is saying, basically, that the expected value of $t$ given input value of $\mathbf{x}$ is equal to $t$ weighted by the conditional probability density function $p(t|\mathbf{x})$.

Next consider a data set of inputs $\mathbf{X} = \{\mathbf{x}_1,\ldots,\mathbf{x}_N\}$ with corresponding target values $t_1,\ldots,t_N$. 

Group these target values $\{t_n\}$ into a column vector denoted by $\mathfrak{t}$, where the typeface is chosen to distinguish it from a single observation of a multivariate target, which would be denoted $\mathbf{t}$.   Assuming data points are drawn independently, the likelihood function is given by (i.e., the product of the probabilities for each individual $t$):

$$p(\mathfrak{t} | \mathbf{X},\mathbf{w},\beta) = \prod_{n=1}^N \mathcal{N} (t_n | \mathbf{w}^T \boldsymbol{\phi}(\mathbf{x}_n), \beta^{-1})$$

where $y(\mathbf{x},\mathbf{w})$ is expressed in terms of the basis functions as $\mathbf{w}^T \boldsymbol{\phi}(\mathbf{x}_n)$.

Here Bishop makes the note that the $\mathbf{x}$ (independent variable) are not modeled.  They are so-called conditioning variables, and so they are dropped from the notation; however, they are still there implicitly. So $p(\mathfrak{t} | \mathbf{x},\mathbf{w},\beta)$ can be  written $p(\mathfrak{t} | \mathbf{w},\beta)$, for example.

Taking the logarithm we obtain (this should all be very familiar by now)

$$\ln p(\mathfrak{t} | \mathbf{w},\beta) = \sum_{n=1}^N \ln \mathcal{N} (t_n | \mathbf{w}^T \boldsymbol{\phi}(\mathbf{x}_n),\beta^{-1})$$
$$ = \frac{N}{2} \ln \beta - \frac{N}{2} \ln (2 \pi) -\beta E_D (\mathbf{w})$$

where the sum-of-squares error function (i.e., the data $t_n$ minus the linear regression model $\mathbf{w}^T \boldsymbol{\phi}(\mathbf{x}_n)$) is given by

$$E_D(\mathbf{w}) = \frac{1}{2} \sum_{n=1}^N \{t_n - \mathbf{w}^T \boldsymbol{\phi}(\mathbf{x}_n) \}^2.$$

To here, this is exactly the same as we did before (when discussing curve fitting above), but now we are going to actually maximize with respect to $\mathbf{w}$, the linear fit coefficients. 

As before, the first two terms are independent of $\mathbf{w}$, so they don't contribute to the maximization. So the minimization reduces to minimizing $\beta$ times the error function.  So the gradient with respect to $\mathbf{w}$ is

$$\nabla \ln p(\mathfrak{t} | \mathbf{w},\beta) = \beta \sum_{n=1}^N \{t_n-\mathbf{w}^T \boldsymbol{\phi}(\mathbf{x}_n)\} \boldsymbol{\phi}(\mathbf{x}_n)^T.$$

Setting this equal to zero gives

$$0 = \sum_{n=1}^{N} t_n \boldsymbol{\phi}(\mathbf{x}_n)^T - \mathbf{w}^T \left (\sum_{n=1}^{N} \boldsymbol{\phi}(\mathbf{x}_n) \boldsymbol{\phi}(\mathbf{x}_n)^T \right ).$$

Solving for $\mathbf{w}$ (which are, recall, the linear coefficients that we are intereted in) we obtain

$$\mathbf{w}_{ML} = (\boldsymbol{\Phi}^T \boldsymbol{\Phi})^{-1} \boldsymbol{\Phi}^T \mathfrak{t}.$$

This is a set of equations known as the normal equations for the least squares problem. Here, $\boldsymbol{\Phi}$ is an $N \times M$ matrix, called the _design matrix_, whose elements are given by $\Phi_{nj} = \phi_j(\mathbf{x}_n)$, so that (and recalling that the $\phi$ are the basis functions):

$$\boldsymbol{\Phi} = \begin{pmatrix} \phi_0(\mathbf{x}_1) &
\phi_1(\mathbf{x}_1) & \cdots & \phi_{M-1}(\mathbf{x}_1) \\
\phi_0(\mathbf{x}_2) & \phi_1(\mathbf{x}_2) & \cdots &
\phi_{M-1}(\mathbf{x}_2) \\
\vdots & \vdots &  & \vdots \\
\phi_0(\mathbf{x}_N) & \phi_1(\mathbf{x}_N) & \cdots & \phi_{M-1}
(\mathbf{x}_N) \\
\end{pmatrix}$$

So the design matrix is a matrix of the individual basis functions evaluated at the $N$ data points $\mathbf{x}_n$.

The quantity

$\boldsymbol{\Phi}^\dagger \equiv (\boldsymbol{\Phi}^T \boldsymbol{\Phi} )^{-1} \boldsymbol{\Phi}^T$ is known as the _Moore-Penrose  pseudo-inverse_ of the matrix $\boldsymbol{\Phi}$. If $\boldsymbol{\Phi}$ is square and invertible, then $\boldsymbol{\Phi}^\dagger \equiv \boldsymbol{\Phi}^{-1}$.

One can also use $E_D(\mathbf{w})$ to investigate the role of the bias parameter $w_0$, i.e., the constant offset. Writing $E_D(\mathbf{w})$ so that $w_0$ is explicit (remembering that $\phi_0 = 1$):

$$E_D(\mathbf{w}) = \frac{1}{2} \sum_{n=1}^N \left [t_n - w_0 - \sum_{j=1}^{M-1} w_j \phi_j (\mathbf{x}_n)\right ]^2.$$

Setting the derivative with respect to $w_0$ equal to zero, and solving for $w_0$, one obtains:

$$w_0= \bar{t} - \sum_{j=1}^{M-1} w_j \bar{\phi_j}$$

where

$$\bar{t} = \frac{1}{N} \sum_{n=1}^N t_n$$
and

$$ \bar{\phi_j} = \frac{1}{N} \sum_{n=1}^N \phi_j(\mathbf{x}_n).$$

So, $w_0$ is the difference between the average of the target values of the training set, and the $w_j$ weighted sum of the average of the basis function values.

Finally, you can maximize with respect to the noise precision parameter $\beta$, yielding

$$\frac{1}{\beta_{ML} }= \frac{1}{N} \sum_{n=1}^N \{t_n -
\mathbf{w}_{ML}^T \boldsymbol{\phi}(\mathbf{x}_n)\}^2.$$

Again,this makes sense; $t$ was set up as the output of a deterministic function with additive Gaussian noise which has a precision equal to $\beta$ (recalling that precision is inverse variance). The maximum likelihood estimate of the inverse precision should be the square of the difference between the data from the training set, with the model using the maximum likelihood coefficients.

### Reflections on the maximum likelihood solution

The maximum likelihood solution requires solving $\mathbf{w}_{ML} = (\boldsymbol{\Phi}^T \boldsymbol{\Phi})^{-1} \boldsymbol{\Phi}^T \mathfrak{t}$. 

Bishop notes that this may offer some numerical difficulties if $\boldsymbol{\Phi}^T \boldsymbol{\Phi}$ is close to singular, especially when some of the columns of $\boldsymbol{\Phi}$ are co-linear or nearly co-linear, then the resulting parameters can have large magnitudes. Bishop notes that good old singular value decomposition can be used in these situations.

Another thing that Bishop discusses, which is puzzling to me, is the idea of the use of sequential (also called online) algorithms. Here, the data points are considered one at a time, and the model parameters are updated after each step. In a sense, the posterior is used as a prior, which I thought was a big no-no. We will see an example of this a little later.

However, one such algorithm does seem to probably make sense - sequential gradient decent. In this method, the error function is a sum over data points, $E=\sum_n E_n$, then after evaluation of parameter $n$, the stochastic gradient decent algorithm updates the parameter vector $\mathbf{w}$ using

$$\mathbf{w}^{(\tau+1) }= \mathbf{w}^\tau -\eta \nabla E_n$$

where $\tau$ denotes the iteration number, not a power of the function, and $\eta$ is the learning rate parameter (which will be discussed shortly). For the sum of least squares, this becomes

$$\mathbf{w}^{(\tau+1)} = \mathbf{w}^\tau -\eta (t_n - \mathbf{w}^{(\tau) T} \boldsymbol{\phi}_n) \boldsymbol{\phi}_n$$

where $\boldsymbol{\phi}_n = \boldsymbol{\phi}(\mathbf{x}_n)$. This looks rather like how a typical  spectral fitting routine works.

## Regularization

Now we have a mechanism to solve for the coefficients, either exactly or approximately.  But what about the problem outlined in the example above - that if there are too many basis functions, the fitted model loses its ability to predict values for new values of $x$.  That can be attacked using a concept called _regularization_.

In the examples shown so far, the absolute best fit, i.e., unbiased estimator, will be found, i.e., the _minimum bias estimator_.  

But to control the models, we introduce an additional parameter, and _trade an increase in bias for a reduction in variance_.  So we no longer get the absolute best fit, but rather a constrained best fit.  This might remind you of the Lagrange multiplier problem, and indeed it looks similar.

Let's illustrate this by looking at the example above.

The polyfit function works by minimizing the error function:

$${E}(\mathbf{w})= \frac{1}{2} \sum_{n=1}^N \{y(x_n,\mathbf{w})-t_n\}^2$$

Here, the $t_n$ are the data points, and the $y$ is the model.

Let's define a modified error function:


$$\tilde{E}(\mathbf{w})= \frac{1}{2} \sum_{n=1}^N \{y(x_n,\mathbf{w})-t_n\}^2 + \frac{\lambda}{2} \lVert \mathbf{w} \rVert^2$$

where $\lVert \mathbf{w}\rVert^2 \equiv \mathbf{w}^T \mathbf{w} = w_0^2+w_1^2+\ldots+w_M^2$, and the coefficient $\lambda$ governs the relative importance of the regularization term compared with the sum-of-squares error. Note that the coefficient $w_0$ is often excluded from the regularizer because its inclusion apparently makes the results depend on the choice of origin. 

As you can see, if the values of $\mathbf{w}$, the coeffients, gets too large, then the modified error function will get large too.  So smaller values of the coefficients will be chosen.


### Regularized Least Squares

Let us more formally the idea of adding a regularization term to control over-fitting. Then, the total error function to be minimized takes the form:

$$E_D(\mathbf{w}) + \lambda E_W(\mathbf{w})$$

where, as before, $\lambda$ is the regularization coefficient, that controls the relative importance of the data-dependent error $E_D(\mathbf{w})$ and the regularization term $E_W(\mathbf{w})$. As before, one of the simplest forms of the regularizer is the sum of squares of the weight vector

$$E_W(\mathbf{w}) = \frac{1}{2} \mathbf{w}^T \mathbf{w}.$$

Let the error function be the sum of squares one:

$$E_D(\mathbf{w}) = \frac{1}{2} \sum_{n=1}^N \{t_n - \mathbf{w}^T
\boldsymbol{\phi}(\mathbf{x}_n) \}^2.$$

with the regularizer portion.  Then the total error function becomes:

$$\frac{1}{2} \sum_{n=1}^N \{t_n - \mathbf{w}^T
\boldsymbol{\phi}(\mathbf{x}_n) \}^2+ \frac{\lambda}{2} \mathbf{w}^T
\mathbf{w}.$$

In statistics this method is called _parameter shrinkage_, since it shrinks parameter values towards zero. The advantage of this form is that the error function remains a quadratic form of $\mathbf{w}$, so the minimum can be found in closed form. Specifically, taking the derivative of the equation above with respect to $\mathbf{w}$ and setting equal to zero yields

$$\mathbf{w} = (\lambda \mathbf{I} + \boldsymbol{\Phi}^T \boldsymbol{\Phi})^{-1} \boldsymbol{\Phi}^T \mathfrak{t}.$$

Note the strong similarity to the least squares solution.

A more general form of the regularizer is given by:

$$\frac{1}{2} \mathbf{w}^T \mathbf{w} + \frac{\lambda}{2} \sum_{j=1}^M
|w_j|^q$$

where the equation above corresponds to the case of $q=2$, and is called **ridge regression**.

The case of $q=1$ is known as the **LASSO regularizer**.  LASSO stands for "least absolute shrinkage and selection". It has the property that if $\lambda$ is sufficiently large, some of the $w_j$ are driven to zero, leading to a sparse model in which some of the basis functions play no role. To see how this works, note that minimizing the above equation is equivalent to minimizing the regular sum-of-squares, subject to the constraint that

$$\sum_{j=1}^M |w_j|^q \leq \eta$$

for an appropriate value of $\eta$, and this type of problem is solved using Lagrange multipliers. 

The figure below from Ivezic (Figure 8.3) illustrates schematically what happens during these regressions.  The unconstrained best fit (i.e., minimum error function) is in the center of the contours.  But the regularization requires the solution to be along a circle a certain distance from the origin.  Here, $|\Theta^2|$ is the sum of the squares of the regression coefficients for the ridge regression (left) and the sum of the absolute values of the regression coefficients for the LASSO regression. 

![Ivezic Figure 8.3](http://www.astroml.org/_images/fig_lasso_ridge_1.png)

The figure below from Ivezic (Figure 8.4) illustrates the results of using regularized regression.  Here, the simulated data is meant to be a model of distance modulus versus redshift (see Ivezic section 8.1.1 for a description).  The basis is a set of Gaussian functions with width $\sigma=0.2$ centered at 100 regular intervals between $0 \leq z\leq 2$.  The lower panels show the fitted weights (coefficients).  The left column shows the result with no regularization, producing weights $w$ that are on the order of $10^{12}$; overfitting is evident. The middle column shows ridge regression with $\lambda=0.005$, and the right column shows LASSO regressoin with $\lambda=0.005$.  

![Ivezic figure 8.4](http://www.astroml.org/_images/fig_rbf_ridge_mu_z_1.png)

The LASSO regression seems particularly nice, as many of the Gaussians end up having zero amplitude.

## Example

We will look at the example above and try to perform ridge regression on it.

First, we'll set up the data again.


In [ ]:
# Set up an x-vector
x=np.sort(np.random.uniform(0,1,11))

# Set up an x-vector with the same range, but sampled more finely
x2=np.linspace(0,1,101)

# y is a sine wave
y=np.sin(2*np.pi*x)

#give y some uncertainty, i.e., apply a Gaussian scatter around 
#the theoretically determined values.
ynew=y+np.random.normal(0,0.15,11)

#assign some errors.  These errors don't have any physical interpretation, 
#but are reasonably appropriate, as they are scattered around the 
#sigma of the uncertainty gaussian (=0.15)
err=np.random.uniform(0.1,0.2,11)

#plot ideal y, and data.
plt.plot(x,y)
plt.errorbar(x, ynew, yerr=err, fmt='o')

In [ ]:
def gaussian_basis(x, mu, sigma):
    return np.exp(-0.5 * ((x - mu) / sigma) ** 2)

centers = np.linspace(0, 1.0, 11)
widths = 0.05
X = gaussian_basis(x[:, None], centers, widths)


In [ ]:
print X.shape
from sklearn.linear_model import LinearRegression, Ridge, Lasso




In [ ]:
## Apply a Gaussian Basis Regression with no regularization

clf = LinearRegression(fit_intercept=True)
clf.fit(X, y)
w = clf.coef_
fit = clf.predict(gaussian_basis(x[:, None], centers, widths))
fit2 = clf.predict(gaussian_basis(x2[:,None],centers,widths))



In [ ]:
plt.plot(x,y)
plt.errorbar(x, ynew, yerr=err, fmt='o')
plt.plot(x,fit)
plt.plot(x2,fit2)
print fit.shape

In [ ]:
## Apply a ridge regression on the gaussian basis.

clf1 = Ridge(fit_intercept=True, alpha=0.1)
clf1.fit(X, y)
w1 = clf1.coef_
fit1 = clf1.predict(gaussian_basis(x[:, None], centers, widths))
fit1_2 = clf1.predict(gaussian_basis(x2[:,None],centers,widths))


In [ ]:
plt.plot(x,y)
plt.errorbar(x, ynew, yerr=err, fmt='o')
plt.plot(x,fit1)
plt.plot(x2,fit1_2)
print fit.shape

In [ ]:
## Apply a LASSO regression on the gaussian basis.

clf2 = Lasso(fit_intercept=True, alpha=0.01)
clf2.fit(X, y)
w2 = clf2.coef_
fit2 = clf2.predict(gaussian_basis(x[:, None], centers, widths))
fit2_2 = clf2.predict(gaussian_basis(x2[:,None],centers,widths))


In [ ]:
plt.plot(x,y)
plt.errorbar(x, ynew, yerr=err, fmt='o')
plt.plot(x,fit2)
plt.plot(x2,fit2_2)


In [ ]:
plt.plot(w)
plt.plot(w1)
plt.plot(w2)

### Choosing the Regularization Parameter $\lambda$

The astute student will recognize that it may seem as though we have traded one hard problem for another.  Instead of having to determine how many basis functions to include, we need to determine what the regularization parameter should be.

First note that if we increase $\lambda$, we increase the constraints on the regression coefficients; this makes sense from the form of the modified error function:

$$\tilde{E}(\mathbf{w})= \frac{1}{2} \sum_{n=1}^N \{y(x_n,\mathbf{w})-t_n\}^2 + \frac{\lambda}{2} \lVert \mathbf{w} \rVert^2$$

But increasing $\lambda$ will likely increase bias (provide a poorer fit).  

Ivezic notes that this question can be addressed using cross validation; we will discuss them momentarily.



### Generalization to Multiple Outputs


Bishop briefly covers the generalization to multiple outputs. That is, instead of producing a single target variable $t$, one may want to produced multiple target variables, i.e., $\mathbf{t}$. Bishop points out that one could treat each component of $\mathbf{t}$ separately, with a separate set of basis functions, but it is also possible to use the same set of basis functions so that:

$$\mathbf{y}(\mathbf{x},\mathbf{w}) = \mathbf{W}^T \boldsymbol{\phi}(\mathbf{x})$$

where $\mathbf{y}$ is a $K$-dimensional column vector, $\mathbf{W}$ is a $M \times K$ matrix of parameters, and $\boldsymbol{\phi}(\mathbf{x})$ is an $M$-dimensional column vector with elements $\phi_j(\mathbf{x})$ with $\phi_0(\mathbf{x}=1)$ as before.

I'll skip all the usual steps (write as Gaussian, take the log, minimize) to produce:

$$\mathbf{W}_{ML} = (\boldsymbol{\Phi}^T \boldsymbol{\Phi})^{-1} \boldsymbol{\Phi}^T \mathbf{T},$$

where $\mathbf{T}$ is a matrix of size $N\times K$ where each $n^{th}$ row is given by $\mathbf{t}_n^T$, for a set of $\mathbf{t}_1,\ldots,\mathbf{t}_N$ observations of the multiple-output training set.

But then Bishop notes that, for each target variable $t_k$, one obtains

$$\mathbf{w}_k=(\boldsymbol{\Phi}^T \boldsymbol{\Phi})^{-1} \boldsymbol{\Phi}^T \mathfrak{t}_k = \boldsymbol{\Phi}^\dagger \mathfrak{t}_k,$$

where $\mathfrak{t}_k$ is an N-dimensional column vector with components $t_{nk}$ for $n=1,\ldots,N$. So the solution of the regression problem decouples between different target variables, and only one pseudo-inverse matrix $\boldsymbol{\Phi}^\dagger$ has to be computed.

Bishop then notes that the further extention to general Gaussian distributions where there are arbitrary covariance matrices is straightforward (and the proof is left as an exercise!).

## Overfitting, Underfitting, and Cross-Validation

As noted after the motivating example, it is important to recognizse the problems and pitfalls that can be associated wtih regression.  A nice discussion is given in Ivezic 8.11.

For simplicity, consider a one-dimensional model with homoscedastic errors. The observed data is $x_i$, and the goal is to predict the value of the target or dependent variable $y_i$. There is a training sample for which both values are observed at every point, and an unknown sample for which only the $x_i$ are measured.

Let's define some sample data where $x$ and $y$ obey

$$0 \leq x_i \leq 3,$$
$$ y_i = x_i \sin(x_i) + \epsilon_i,$$

where the noise has been drawn from a normal distribution, i.e., $\epsilon_i \sim \mathcal{N}(0,0.1)$. The figure below (Figure 8.12 from Ivezic) shows the example for 20 regularly spaced values of $x_i$, with a linear model.  (This is quite comparable to the motivating example above).

![Ivezic figure 8.12](http://www.astroml.org/_images/fig_cross_val_A_1.png)

It is quite clear that the straight line does not provide a good fit: the data are _underfit_.  It can also be said to suffer from _high bias_.

Likewise, a high order polynomial $d=19$ _overfits_ the data; it suffers from _high variance_.

![Ivezic figure 8.13](http://www.astroml.org/_images/fig_cross_val_B_1.png)


### Cross-Validation

Cross-validation is a method to quantitatively evaluate the bias and variance of a regression model.

There are several different approaches to cross validation. Ivezic goes through one in detail, and mentions others briefly.

The simplest one is to split the training data into three sets: the training set, the cross-validation set, and the test set. As a rule of thumb, the training set should comprise 50-70% of the original training data, while the other two catagories are divided equally.

The training set is used (obviously) to derive the estimated values of $\mathbf{w}$, i.e., $\mathbf{w}_{ML}$. Then, using the training set, the training error $\epsilon_{tr}$ is determined using the equation above (except one would get the squared difference between the training target values $t_n$ and the model values $\mathbf{w}^T\phi(\mathbf{x}_n)$ - the equation above is just for a linear model).

Next, one would obtain the cross-validation error $\epsilon_{cv}$ by evaluating the error function using the cross-validation set and the parameter values derived using the training set. As we have seen, from the example above, that error will be high if the model is overfit. So we can conclude that the model minimizes the cross-validation error is the best one.

What, then, is the use of the test set on top of the training set and the cross-validation set? Ivezic argues that, just as the parameters $\mathbf{w}$ are learned from the training set, hyperparameters such as the order $d$ of the polynomial are learned from the cross-validation set. So the hyperparameters can be overfit to the cross-validation data, and the cross-validation error can give an overly optimistic view of the performance of the model on a new data set. So the test error gives a better representation of the error on a new data set.

Ivezic Figure 8.14 (top panel) shows the training erorr and cross-validation error as a function of the polynomial degree $d$. For small $d$, the training and cross validation error are both high - the data is underfit by the model. For large $d$ the training error becomes small; the model is starting to fit the noise. But the cross validation error becomes large, indicating that the data is over fit by the model.

![Ivezic Figure 8.14](http://www.astroml.org/_images/fig_cross_val_C_1.png)

Let's try this with the data from the motivating example.  We'll create three data sets (training, test, and cross-validation) using the same sine generating function, but we'll let the error be different (i.e., separate draws) for each data set.  In addition, we'll have the X-values that each one is evaluated at be different.

In [ ]:
plt.plot(x,y)

x_train=np.sort(np.random.uniform(0,1,11))
x_test=np.sort(np.random.uniform(0,1,11))
x_cv=np.sort(np.random.uniform(0,1,11))

y_train=np.sin(2*np.pi*x)

ynew_train=np.sin(2*np.pi*x_train)+np.random.normal(0,0.2,11)
ynew_test=np.sin(2*np.pi*x_test)+np.random.normal(0,0.2,11)
ynew_cv=np.sin(2*np.pi*x_cv)+np.random.normal(0,0.2,11)

plt.plot(x_train,ynew_train,'o')
plt.plot(x_test,ynew_test,'o')
plt.plot(x_cv,ynew_cv,'o')


In [ ]:
# Train on the training set

result_0=np.polyfit(x_train,ynew_train,0)
result_1=np.polyfit(x_train,ynew_train,1)
result_2=np.polyfit(x_train,ynew_train,2)
result_3=np.polyfit(x_train,ynew_train,3)
result_4=np.polyfit(x_train,ynew_train,4)
result_5=np.polyfit(x_train,ynew_train,5)
result_6=np.polyfit(x_train,ynew_train,6)
result_7=np.polyfit(x_train,ynew_train,7)
result_8=np.polyfit(x_train,ynew_train,8)
result_9=np.polyfit(x_train,ynew_train,9)
result_10=np.polyfit(x_train,ynew_train,10)


In [ ]:
numpnts=np.float(x.shape[0])
print numpnts
y0=np.poly1d(result_0)
y1=np.poly1d(result_1)
y2=np.poly1d(result_2)
y3=np.poly1d(result_3)
y4=np.poly1d(result_4)
y5=np.poly1d(result_5)
y6=np.poly1d(result_6)
y7=np.poly1d(result_7)
y8=np.poly1d(result_8)
y9=np.poly1d(result_9)
y10=np.poly1d(result_10)


err_train=np.zeros(11)

err_train[0]=(1.0/numpnts)*(((ynew_train-y0(x_train))**2)).sum()
err_train[1]=(1.0/numpnts)*(((ynew_train-y1(x_train))**2)).sum()
err_train[2]=(1.0/numpnts)*(((ynew_train-y2(x_train))**2)).sum()
err_train[3]=(1.0/numpnts)*(((ynew_train-y3(x_train))**2)).sum()
err_train[4]=(1.0/numpnts)*(((ynew_train-y4(x_train))**2)).sum()
err_train[5]=(1.0/numpnts)*(((ynew_train-y5(x_train))**2)).sum()
err_train[6]=(1.0/numpnts)*(((ynew_train-y6(x_train))**2)).sum()
err_train[7]=(1.0/numpnts)*(((ynew_train-y7(x_train))**2)).sum()
err_train[8]=(1.0/numpnts)*(((ynew_train-y8(x_train))**2)).sum()
err_train[9]=(1.0/numpnts)*(((ynew_train-y9(x_train))**2)).sum()
err_train[10]=(1.0/numpnts)*(((ynew_train-y10(x_train))**2)).sum()

err_test=np.zeros(11)

err_test[0]=(1.0/numpnts)*(((ynew_test-y0(x_test))**2)).sum()
err_test[1]=(1.0/numpnts)*(((ynew_test-y1(x_test))**2)).sum()
err_test[2]=(1.0/numpnts)*(((ynew_test-y2(x_test))**2)).sum()
err_test[3]=(1.0/numpnts)*(((ynew_test-y3(x_test))**2)).sum()
err_test[4]=(1.0/numpnts)*(((ynew_test-y4(x_test))**2)).sum()
err_test[5]=(1.0/numpnts)*(((ynew_test-y5(x_test))**2)).sum()
err_test[6]=(1.0/numpnts)*(((ynew_test-y6(x_test))**2)).sum()
err_test[7]=(1.0/numpnts)*(((ynew_test-y7(x_test))**2)).sum()
err_test[8]=(1.0/numpnts)*(((ynew_test-y8(x_test))**2)).sum()
err_test[9]=(1.0/numpnts)*(((ynew_test-y9(x_test))**2)).sum()
err_test[10]=(1.0/numpnts)*(((ynew_test-y10(x_test))**2)).sum()

plt.plot(err_test[0:7])


err_cv=np.zeros(11)

err_cv[0]=(1.0/numpnts)*(((ynew_cv-y0(x_cv))**2)).sum()
err_cv[1]=(1.0/numpnts)*(((ynew_cv-y1(x_cv))**2)).sum()
err_cv[2]=(1.0/numpnts)*(((ynew_cv-y2(x_cv))**2)).sum()
err_cv[3]=(1.0/numpnts)*(((ynew_cv-y3(x_cv))**2)).sum()
err_cv[4]=(1.0/numpnts)*(((ynew_cv-y4(x_cv))**2)).sum()
err_cv[5]=(1.0/numpnts)*(((ynew_cv-y5(x_cv))**2)).sum()
err_cv[6]=(1.0/numpnts)*(((ynew_cv-y6(x_cv))**2)).sum()
err_cv[7]=(1.0/numpnts)*(((ynew_cv-y7(x_cv))**2)).sum()
err_cv[8]=(1.0/numpnts)*(((ynew_cv-y8(x_cv))**2)).sum()
err_cv[9]=(1.0/numpnts)*(((ynew_cv-y9(x_cv))**2)).sum()
err_cv[10]=(1.0/numpnts)*(((ynew_cv-y10(x_cv))**2)).sum()

In [ ]:
plt.axis([0,11,-0.10,1.5],fontsize=20,linewidth=4)


plt.plot(err_train[0:11])
plt.plot(err_test[0:11])
plt.plot(err_cv[0:11])

### Learning Curves

Say one has fit some model to data, but the result is not satisfactory because the cross validation error is much larger than the training error. How can it be improved?

- **Get more training data**. Sometimes this works but not always (see below)

- **Use a more / less complicated model**. The complexity of the model should be chosen as a balance between the bias (training model error, basically) and the variance (cross-validation error, basically).

- **Use more or less regularization**. In general, increasing regularization has a similar effect as decreasing the model complexity - we saw this directly for the LASSO model.

- **Increase the number of features**. Adding more observations of each object in the set. This would be like, if you had $ugvri$ photometry, and you added $J$, $H$, and $K$, for example.

A _learning curve_ can be used to determine the number of training data required.  A learning curve plots the training error and cross-validation error as a function of the number of training points.

Let the model be represented by the set of parameters $\boldsymbol{\theta}$ (denoted by $\mathbf{w}$ in Bishop). So, $\boldsymbol{\theta} = \{\theta_0, \theta_1, \ldots, \theta_d\}$. Let $\boldsymbol{\theta}^{(n)}$ denote the model parameters that best fit the first $n$ data points, i.e., $\boldsymbol{\theta}^{(n)} = \{ \theta_0^{(n)}, \theta_1^{(n)},\ldots,\theta_d^{(n)}\}$. So the truncated training error is:

$$\epsilon_{tr}^{(n)} = \sqrt{\frac{1}{n} \sum_{i=1}^n \left [ y_i - \sum_{m=0}^d \theta_0^{(n)} x_i^m \right ]^2}. $$

Note that the training error $\epsilon_{tr}^{(n)}$ is evaluated using only the $n$ points that the model parameters $\theta^{(n)}$ were trained on, not the full set of training data.

The cross validation error is then given by:

$$\epsilon_{cv}^{(n)} = \sqrt{\frac{1}{n} \sum_{i=1}^{N_{cv}} \left [ y_i - \sum_{m=0}^d \theta_0^{(n)} x_i^m \right ]^2}, $$

i.e., it is evaluated over the whole cross validation set. The learning curve is then these two errors as a function of number of points $n$ in the training set. Ivezic Figure 8.15 shows these values for $d=2$ and $d=3$ for the simulated data above.

![ivezic fig 8.15](http://www.astroml.org/_images/fig_cross_val_D_1.png)

These plots are very illuminating. These are some common features.

- As we increase the size of the training set, the training error increases. That makes sense; more data will be more difficult to fit with a simple model. For $N \leq \sim 15$, the cross-validation error is very high, but it levels off for $N \geq \sim 15$.
- As we increase the size of the training set, the cross-validation error decreases. A small data set is less representative and more easily over-fit than a large data set.
- The training error is less than the cross validation error; this should always be true.
- As the number of data become large, the training and cross-validation errors would converge. At that point, the model error is dominated by bias, and additional training data cannot improve the results for that particular model.

As is seen by this example, learning curves seem very useful for evaluating a model, and giving hints for how the model/fit can be improved. There are two possible situations;

1. **The training error and cross-validation error have converged**. In this case, increasing the number of training points is futile as the model is dominated by bias (i.e., it is underfitting the data). 
For a high bias model, the following approaches may help.
   - Add additional features to the data
   - Increase model complexity.
   - Decrease regularization
2. **The training error is much smaller than the cross-validation error**. Increasing the number of training points is likely to improve the model. This indicates a model error dominated by variance, and the data is overfit. For a high-variance model, the following approaches may help:  
    - increase the training set size.
    - decrease model complexity.
    - increase regularization.

### Other Cross-Validation Techniques

There are numerous other cross-validation techniques that are suitable for different situations. Some are briefly described in Ivezic.

**Twofold Cross-validation**  (Note that description in Ivezic doesn't make sense, and I suspect typos, so I am taking the wikipedia definition). For each fold, we randomly assign data points to two sets $d_0$ and $d_1$, so that both sets are equal size (this is usually implemented by shuffling the data array and then splitting it in two). We then train on $d_0$ and test on $d_1$, followed by training on $d_1$ and testing on $d_0$.

**K-fold Cross-validation**  A generalization of the two-fold case. Divide the data into $K+1$ subsets, including the training set $d_0$ and the cross-validation sets $d_1,d_2,\ldots,d_k$. Train $k$ different models, each time leaving out a subset to measure the cross-validation error. The final training error and cross-validation error can be computed using the mean or median of the set of results.

**Leave-one-out Cross-validation** We have already mentioned this method. Repeatedly train the model on the training data, but each time, leave one datum out. The final training error and cross-validation error are estimated using the mean or median of the individual trials. This method can be useful when the data set is small.

**Random subset cross-validation** The cross-validation set and the training set are selected by randomly partitioning the data and repeating a number of ties until the error statistics are well sampled. 

## Measurement Errors and Upper Limits in Regression

So far, all we have worked with in terms of errors has been the precision $\beta$, and that has generally been assumed to be homoscedastic and known. But real astronomical data isn't like
that. E.g.: 

- There may be heteroscedastic errors on the target variables.
- There may be errors on the $x$ values.
- There may be upper or lower limits.
- There may be selection effects.

Ivezic discusses the case of uncertainties in the independent variable briefly in Section 8.8 using the total least squares treatment. They do not, however, (and surprisingly enough) discuss censored data, i.e., data with upper and lower limits. These data are useful because there is some information in a limit. But they can't be treated directly with the other data.

The venerable software packages for dealing with upper limits is the ASURV package found [here](http://astrostatistics.psu.edu/statcodes/sc_censor.html).  It is part of the Penn State StatCodes statistical software website.

However, it seems that all of this may have been subsumed by a single paper ["Some Aspects of Measurement Error in Linear Regression of Astronomical Data" by Brandon Kelly (ApJ, 665, 1489 (2007)](http://adsabs.harvard.edu/abs/2007ApJ...657..116K).   (This paper have 409 citations as of 10/18/2017.)  

The package described was developed in IDL as "linmix_err" and distributed through the IDL Astronomy User's Library. There is a python distribution found on github [here](https://github.com/jmeyers314/linmix).  

This seems to me to be the state of the art for linear regression for astrophysics, although there are some significant limitations:
- The python distribution only treats a single variable (i.e., you need to be fitting just $x$ and $y$).  This is not a limitation of the code; but rather a limitation of the port.
- The model is limited to a fitting a line, i.e., $y=ax+b$.  But it also outputs the correlation coefficient.

So the applications you might want to consider are: are the data correlated, and what is the slope of the correlation (keeping in mind that multiple variables are not allowed).

First, some notation (Section 2):

- A parameter that is an estimate has a hat on it, e.g., $\hat{\theta}$.
- Greek letters denote the true values, roman letters denote the contaminated measured value.
- The bias of an estimator is $E(\hat{\theta}) - \theta_0$ where $E(\hat{\theta})$ is the expectation value of the estimator $\hat{\theta}$, and $\theta_0$ is the true value. An unbiased estimator is one where $E(\hat{\theta}) = \theta_0$.
- As usual, the Gaussian distribution is $\mathcal{N}(\mu,\sigma^2)$ for a single variable, while $\mathcal{N}_p(\boldsymbol{\mu},\boldsymbol{\Sigma})$ is the multivariate case with $p$-element vector $\boldsymbol{\mu}$ and $p \times p$ covariance matrix $\boldsymbol{\Sigma}$. Also $\mathcal{N}(x|\mu,\sigma^2)$ makes it a function of $x$.
- a $\sim$ means "is drawn from" or "is distributed as"

### Effect of Measurement Error on Correlation and Regression

Section 3 of the paper offers a review of the effect of measurement error on regression slope. As noted above, Kelly (2007) also present the results for correlation coefficient.

Let the independent variable be $\boldsymbol{\xi}$ and the dependent variable be $\boldsymbol{\eta}$. Assume that $\boldsymbol{\xi}$ is an $n$-point vector of data points drawn randomly from some probability distribution. Then

$$\eta_i = \alpha + \beta \xi_i + \epsilon_i,$$

where $\epsilon_i$ is a random variable representing the intrinsic scatter in $\eta_i$ about the regression relationship and $(\alpha,\beta)$ are regression coefficients. The mean of $\boldsymbol{\epsilon}$ is assumed to be zero, and the variance is assumed to be constant and is denoted as $\sigma^2$.

We do not observe the actual values of $(\boldsymbol{\xi},\boldsymbol{\eta})$ but rather observe the values $(\mathbf{x},\mathbf{y})$ which are measured with error. The measured values are related to the actual values as

$$x_i = \xi_i +\epsilon_{x,i},$$
$$y_i = \eta_i + \epsilon_{y,i},$$

where $\epsilon_{x,i}$ and $\epsilon_{y,i}$ are the random measurement errors on $x_i$ and $y_i$. The errors are normally distributed with known variances $\sigma^2_{x,i}$ and $\sigma^2_{y,i}$ and covariance $\sigma_{xy,i}$. For simplicity, during this section, assume that $\sigma_x^2$, $\sigma_y^2$ and $\sigma_{xy}$ are the same for each data point; this assumption will ultimately be lifted later.

When the data are measured without error, the least-squares estimate of the regression slope, $\hat{\beta}_{OLS}$ is

$$\hat{\beta}_{OLS} =
\frac{{Cov}(\boldsymbol{\xi},\boldsymbol{\eta})}{{Var}(\boldsymbol{\xi})},
$$

$$\hat{\rho} =
\frac{{Cov}(\boldsymbol{\xi},\boldsymbol{\eta})}
{\sqrt{{Var}(\boldsymbol{\xi})
{Var}(\boldsymbol{\eta})}} = \hat{\beta}_{{OLS}}
\sqrt{\frac{{Var}(\boldsymbol{\xi})}{{Var}(\boldsymbol{\eta})}},$$

where ${Cov}(\boldsymbol{\xi},\boldsymbol{\eta})$ is the sample covariance between $\boldsymbol{\xi}$ and $\boldsymbol{\eta}$, and ${Var}(\boldsymbol{\xi})$ is the sample variance of $\boldsymbol{\xi}$.  Here, $\rho$ is the correlation coefficient, which is also estimated. This is a rather standard result, e.g., see Wikipedia on ["simple linear regression"](https://en.wikipedia.org/wiki/Simple_linear_regression).

When the data are measured with error, the slope becomes instead:

$$\hat{\beta}_{OLS} = \frac{{Cov}(\mathbf{x},\mathbf{y})} {{Var}(\mathbf{x})} = \frac{{Cov}(\boldsymbol{\xi},\boldsymbol{\eta})+\sigma_{xy}}
{{Var}(\boldsymbol{\xi}+\sigma_x^2)}. $$

$$\hat{r} =
\frac{{Cov}(\mathbf{x},\mathbf{y})}{\sqrt{{Var}(\mathbf{x}) {Var}(\mathbf{y})}}
= \frac{{Cov}(\boldsymbol{\xi},\boldsymbol{\eta})+
\sigma_{xy}} {\sqrt{[{Var}(\boldsymbol{\xi}) +
\sigma_x^2][{Var}(\boldsymbol{\eta}) + \sigma_y^2]}}.$$

So you can see that the slope is biased by the error.

One can assess the effect of measurement error in terms of the ratios $R_x = \sigma_x^2/{Var}(\mathbf{x})$, $R_y = \sigma_y^2/{Var}(\mathbf{y})$, $R_{xy} = \sigma_{xy}/{Cov}(\mathbf{x},\mathbf{y})$ as follows:

$$\frac{\hat{b}}{\hat{\beta}} = \frac{1-R_x}{1-R_{xy}}.$$

$$\frac{\hat{r}}{\hat{\rho}} = \frac{\sqrt{(1-R_x)(1-R_y)}}{1-R_{xy}}.$$

So the effect of measurement error on the slope looks to me like it depends on how the covariance compares with the variance and how those compare with the error. Covariance is $\bar{xy} - \bar{x}\bar{y}$ and variance is $\bar{x^2} - \bar{x}^2$. The ratios are a kind of excess variance, i.e., the variance minus the measurement error squared.  It is noted in the paper that the presence of covariate measurement error reduces the magnitude of the observed correlation, as well as biasing the slope estimate toward zero.

Figure 1 in the paper plots the fractional bias in the correlation coefficient, $(\hat{\rho} - \hat{r})/\hat{\rho}$ as a function of $R_x$ and $R_y$ when the errors are uncorrelated. Basically, this shows that the larger the measurement error relative to the variance, the lower the correlation coefficient.

### The Statistical Model

Section 4 describes the statistical model. Knowing what we know now, this is not actually that bad. One thing to note that it is a hierarchical problem, so repeated use of Bayes' Theorem is made, along with product rule.

They first consider 1 independent variable. Here are the assumptions:

- The independent variable $\xi$ is drawn from a probability distribution $p(\xi | \boldsymbol{\psi})$, where $\boldsymbol{\psi}$ denotes the parameters for the probability distribution.
- The dependent variable is drawn from the conditional distribution of $\eta$ given $\xi$, denoted as $p(\eta | \xi,\boldsymbol{\theta})$; $\boldsymbol{\theta}$ denotes the parameters of this distribution.
- The joint distribution of $\xi$ and $\eta$ is then $p(\xi,\eta | \boldsymbol{\psi}, \boldsymbol{\theta})$.
- The data are scattered around the true distribution  with a normal distribution, so $p(\eta |  \xi,\boldsymbol{\theta})$ has a normal distribution with mean $\alpha + \beta \xi$ and variance $\sigma^2$, and $\boldsymbol{\theta} = (\alpha,\beta,\sigma^2)$.

So this application is rather limited in scope compared with what we have been looking at so far; they are only fitting a straight line.

The likelihood of the data is given by:

$$p(x,y | \boldsymbol{\theta},\boldsymbol{\psi}) = \int \int p(x,y,\xi,\eta | \boldsymbol{\theta},\boldsymbol{\psi})\, d\xi\, d\eta,$$

where $p(x,y,\xi,\eta | \boldsymbol{\theta},\boldsymbol{\psi)}$ is the complete data likelihood. It is helpful to deconvolve this into conditional probability densities using the product rule:

$$p(x,y | \boldsymbol{\theta},\boldsymbol{\psi)} = \int \int p(x,y | \xi,\eta) p(\eta | \xi, \boldsymbol{\theta}) p(\xi | \boldsymbol{\psi)}\, d\xi\, d\eta.$$

The product inside the above integral comprises $p(x,y | \xi,\eta)$, which is the joint distribution of the measured $x$ and $y$ at the values of $\xi$ and $\eta$, and therefore depend on the distribution of the measurement errors $\epsilon_x$ and $\epsilon_y$. Gaussian measurement error is assumed, so $p(x_i,y_i | \xi_i,\eta_i)$ is a multivariate normal density with mean $\xi_i,\eta_i)$ and covariance matrix $\boldsymbol{\Sigma}_i$, where $\Sigma_{11,i} = \sigma^2_{y,i}$, $\Sigma_{22,i} = \sigma_{x,i}^2$, and $\Sigma_{12,i} = \sigma_{xy,i}$. 

Then the hierarchial statistical model is:

$$\xi_i \sim p(\boldsymbol{\xi} | \boldsymbol{\psi}),$$

$$ \eta_i | \xi_i \sim \mathcal{N}(\alpha + \beta \xi_i,\sigma^2),$$

$$ y_i, x_i | \eta_i, \xi_i \sim \mathcal{N}_2 ([\eta_i,\xi_i],\boldsymbol{\Sigma}_i).$$

The hierachical nature of these equations can be recognized, and it makes sense. 
 - First $\xi_i$ (the independent variable) is drawn from its distribution $p(\boldsymbol{\xi},\boldsymbol{\psi})$.
 - Then $\eta_i$ (the dependent variable), given the drawn value of $\xi_i$, is drawn from a normal distribution, which depends on the intrinsic intercept $\alpha$ and slope $\beta$, as well as the intrinsic scatter in the data, which is also assumed to be a normal distribution with zero mean and $\sigma^2$ for variance.  An illustration of this idea is seen in Bishop figure 1.28.
 - Then you draw the actual observed data, $x_i$ and $y_i$, given the intrinsic values $\xi_i$ and $\eta_i$, and the measurement uncertainty, which is contained in the covariance matrix $\Sigma$.
 
Note that if $x_i$ is measured without error, then $p(x_i | \xi_i)$ is a Dirac $\delta$-function, and $p(x_i, y_i | \xi_i,\eta_i) = p(y_i | \eta_i) \delta(x_i-\xi_i)$. A similar formula applies if $y_i$ is measured without error.

The equation above for $p(x,y | \boldsymbol{\theta},\boldsymbol{\psi})$ is used to obtain the observed data likelihood function for any assumed distribution of $\boldsymbol{\xi}$. (Remember that $\boldsymbol{\xi}$ is the true distribution of the independent variable.) In this paper, $p(\boldsymbol{\xi} | \boldsymbol{\psi})$ is modeled as a _Gaussian mixture model_ with $K$ Gaussian functions.

$$p(\xi_i | \boldsymbol{\psi}) = \sum_{k=1}^K \frac{\pi_k}{\sqrt{2 \pi \tau_k^2}} \exp \left [ - \frac{1}{2} \frac{(\xi_i - \mu_k)^2} { \tau_k^2} \right ],$$

where $\sum_{k=1}^K \pi_k = 1$, and $\pi_k$ is interpreted as the probability of drawing a data point from the $k$th Gaussian function, and that Gaussian function has mean of $\mu_k$ and standard deviation of $\tau_k$. Notation is $\boldsymbol{\pi} = (\pi_1,\ldots,\pi_k)$, $\boldsymbol{\mu}=(\mu_1,\ldots,\mu_k)$, and $\boldsymbol{\tau}^2 = (\tau_1^2,\ldots,\tau_k^2)$, and note that $\boldsymbol{\psi} = (\boldsymbol{\pi},\boldsymbol{\mu},\boldsymbol{\tau}^2)$. 

This relationship is used because it is flexible enough to adapt to a wide variety of distributions, but is also conjugate to the regression relationship and the measurement error distribution (also normally distributed), and so that will make the math easier, because you can make use of the fact that Gaussians add easily.

Having the distribution of $p(\boldsymbol{\xi},\boldsymbol{\psi})$, the measured data likelihood for the $i$th data point can be determined using the double integral equation above. 

Let the measured data point be denoted as $\mathbf{z} = (\mathbf{y},\mathbf{x})$, then the measured data likelihood function for the $i$th point is a mixture of bivariate normal distribuitions with weights $\boldsymbol{\pi}$, means, $\boldsymbol{\zeta} = (\boldsymbol{\zeta}_1, \ldots, \boldsymbol{\zeta}_K)$, and covariance matrices $\mathbf{V}_i = (\mathbf{V}_{1,i},\ldots,\mathbf{V}_{K,i})$, where these latter parameters are defined below. Because the data points are statistically independent, then we can do the usual thing and assume that the probability of all the data points is the product of the probabilities of each one separately, as follows:

$$p(\mathbf{x},\mathbf{y} | \boldsymbol{\theta}, \boldsymbol{\psi}) = \prod_{i=1}^n \sum_{k=1}^K \frac{\pi_k}{2\pi |\mathbf{V}_{k,i} | ^{1/2}} \exp \left [ -\frac{1}{2} (\mathbf{z} - \boldsymbol{\zeta}_k)^T \mathbf{V}^{-1}_{k,i} (\mathbf{z}_i - \boldsymbol{\zeta}_k) \right ],$$

where

$$\boldsymbol{\zeta}_k = (\alpha + \beta \mu_k,\mu_k),$$

$$\mathbf{V}_{k,i} = \begin{pmatrix} \beta^2 \tau_k^2 + \sigma + \sigma_{y,i}^2 & \beta \tau_k^2 + \sigma_{xy,i} \\ \beta \tau_k^2 + \sigma_{xy,i} & \tau_k^2 + \sigma_{x,i}^2 \end{pmatrix}. $$

Let us pause reflect upon the components of this equation. The left hand side is the same as the double integral equation above, so this is the "answer", in general terms. It is clear that the fact that a product of guassians is a gaussian has been used extensively.   

It will useful to decompose the measured data likelihood, $p(x_i,y_i | \boldsymbol{\theta},\boldsymbol{\psi}) = p(y_i | x_i, \boldsymbol{\theta}, \boldsymbol{\psi}) p(x_i, \boldsymbol{\psi})$ for application when the data contain nondetections. Then the marginal likelihood of $x_i$ is

$$p(x_i | \psi) = \sum_{i=1}^K \frac{\pi_k}{\sqrt{2\pi(\tau_k^2 +\sigma_{x,i}^2)}} \exp \left [ -\frac{1}{2} \frac{(x_i - \mu_k)^2}{\tau_k^2 + \sigma_{x,i}^2} \right ],$$

and the conditional distribution of $y_i$ given $x_i$ is

$$p(y_i | x_i, \boldsymbol{\theta}, \boldsymbol{\psi}) = \sum_{k=1}^K \frac{\gamma_k}{\sqrt{2 \pi {Var}(y_i | x_i,k)}} \exp \left \{ -\frac{1}{2} \frac{[y_i-E(y_i | x_i, k)]^2}{{Var} (y_i | x_i,k)} \right \}, $$

where

$$\gamma_k = \frac{\pi_k \mathcal{N}(x_i | \mu_i , \tau_k^2 + \sigma_{x,i}^2)} {\sum_{j=1}^K \pi_j \mathcal{N}(x_i | \mu_j, \tau_j^2 + \sigma_{x,i}^2)}, $$

and

$$E(y_i | x_i, k) = \alpha + \frac{\beta \tau_k^2+\sigma_{xy,i}} { \tau_k^2 + \sigma_{x,i}^2} x_i + \frac{\beta \sigma_{x,i}^2 -\sigma_{xy,i}}{\tau_k^2 + \sigma_{x,i}^2} \mu_k,$$

and

$${Var}(y_i | x_i, k) = \beta^2 \tau_k^2 + \sigma^2 + \sigma^2_{y,i} - \frac{(\beta \tau_k^2 - \sigma_{xy,i})^2} { \tau_k^2 + \sigma_{x,i}}.$$

Here, $\gamma_k$ can be interpreted as the probability that $i$th data point was drawn from the $k$th Gausisan function, given $x_i$.  Recall that we met $\gamma_k$ back in Lecture08, where they were called the _responsibilities_.  $E(y_i|x_i,k)$ gives the expectation of $y_i$ given $x_i$, given that the data point was drawn from the $k$th Gausisan function.  Finally ${Var}(y_i,x_i,k)$ gives the variance in $y_i$ at $x_i$, given that the data point was drawn from the $k$th Gaussian distribution.  

### Relationship between Uniformly Distributed Covariants and Effective $\chi^2$ Estimators

Consider the case where the distribution of $\boldsymbol{\xi}$ is assumed to be uniform, i.e., $p(\boldsymbol{\xi}) \propto 1$. One might consider a uniform distribution "fairer" than the mixture of Gaussians. A uniform distribution for $\boldsymbol{\xi}$ can be obtained by letting $\tau^2 \rightarrow \infty$. Then the likelihood function for $p(\boldsymbol{\xi}) \propto 1)$ can be obtained from the equation for $p(y_i | x_i, \boldsymbol{\theta}, \boldsymbol{\psi})$ above by taking $\tau^2 \rightarrow \infty$ and $K=1$. If measurement errors are uncorrelated, this becomes:

$$p(\mathbf{y} | \mathbf{x}, \boldsymbol{\theta}) = \prod_{i=1}^n \frac{1}{\sqrt{ 2\pi (\sigma^2 +\sigma^2_{y,i} + \beta^2 \sigma^2_{x,i})}} \exp \left [ -\frac{1}{2} \frac{(y_i-\alpha-\beta x_i)^2} {\sigma^2 + \sigma^2_{y,i} + \beta^2 \sigma^2_{x,i}} \right ].$$

This equation is interesting because the argument of the exponent is the so-called FITEXY goodness of fit statistic $\chi^2_{EXY}$ (Tremaine et al. 2002). So, according to the FITEXY method, this quantity (i.e., the argument of the exponent) would be minimized However, minimizing $\chi^2_{EXY}$ is not the same as maximizing the conditional likelihood of $\mathbf{y}$ given $\mathbf{x}$. See the paper for more discussion on this point.



### Regression with Multiple Independent Variables

This formalism can be generalized to multiple independent variables, i.e,.

$$\eta_i = \alpha + \boldsymbol{\beta}^T \boldsymbol{\xi}_i + \epsilon_i,$$ where $\boldsymbol{\beta}$ is now a p-element vector and $\boldsymbol{\xi_i}$ is also a p-element vector which contains the values of the independent variables for the $i$th data point. As before, the distriubtion of $\boldsymbol{\xi}_i$ is approximated using a mixture of $K$ normal distributions, but now they are multivariate. See Kelly 2007 section 4.3 for the relevant equations.  Note that this capability is not implemented in the linmix python distribution.



### Selection Effects

Consider a sample that contains $N$ sources, where $N$ is unknown. A survey is made, and only $n$ are detected because of the survey's selection method. The astronomer is then interested in how both measurement error and the survey's selection method affect the statistical inference.

This is important because even though the objects were not detected, the fact that they were detected down to a certin limit (due to the survey sensitivity) yields some information.  

To manage the nondetections, introduce an **indicator variable** $\mathbf{I}$, which denotes whether a source is included in the sample, i.e., $I_i =1$ if included, $I_i=0$ if not included. Also assume that the selection function only depends on the measured values $\mathbf{x}$ and $\mathbf{y}$, which makes sense for a flux-limited sample. Then, the selection function of the sample is the probability of including a source with a given $\mathbf{x}$ and $\mathbf{y}$, which is $p(\mathbf{I} | \mathbf{x},\mathbf{y})$.

More concretely, consider a flux-limited survey. Then the source will be detected if its measured flux falls above some set flux limit. Then, if a sample is above the flux limit, $p(I_i = 1 | y_i) = 1$, and if it is not, then $p(I_i =1 | y_i) = 0$. _Kelly notes that since the selection function depends on the measured values and not the true values, then sources with true values greater than the limit can be missed in a survey, while those below the flux limit can be detected_. This is called a _Malmquist bias_; we have discussed this before.

Including the indicator variable $\mathbf{I}$, the complete data likelihood becomes

$$p(\mathbf{x},\mathbf{y},\boldsymbol{\xi}, \boldsymbol{\eta}, \mathbf{I} | \boldsymbol{\theta}, \boldsymbol{\psi}) = p(\mathbf{I} | \mathbf{x},\mathbf{y}) p(\mathbf{x},\mathbf{y} | \boldsymbol{\xi},\boldsymbol{\eta}) p(\boldsymbol{\eta}| \boldsymbol{\xi}, \boldsymbol{\theta}) p(\boldsymbol{\xi},\boldsymbol{\psi}).$$

This is rather similar to the equations above, with the inclusion of the additional parameter $\mathbf{I}$. Integrating over the missing data, one finds:

$$p(\mathbf{x}_{obs}, \mathbf{y}_{obs} | \boldsymbol{\theta}, \boldsymbol{\psi}, N) \propto C_n^N \prod_{i \in \mathcal{A}_{obs}} p(x_i,y_i | \boldsymbol{\theta}, \boldsymbol{\psi}) $$

$$ \times \prod_{j \in \mathcal{A}_{mis}} \int p(I_j = 0 | x_j, y_j) p(x_j,y_j | \xi_j, \eta_j) p(\eta_j | \xi_j, \boldsymbol{\theta}) p(\xi_j | \psi)\, d x_j \, dy_j\, d\xi_j \, d\eta_j,$$

where $C_n ^ N$ is the binomial coefficient, necessary to give the number of possible ways of selecting $n$ sources form $N$ sources, and $\mathcal{A}_{obs}$ denotes the set of $n$ included sources, $\mathbf{x}_{obs}$ and $\mathbf{y}_{obs}$ denotes the values of $\mathbf{x}$ and $\mathbf{y}$ fo rthe included sources, while $\mathcal{A}_{mis}$ denotes the $N-n$ missing sources. Note, there are actually other terms in this equation, but they have been omitted for clarity.

Kelly (2007) then breaks down this general case to the sub cases where the selection is based only on the independent variable, and the separate subcase where the selection is based on the dependent variable. Please see the paper for details.

### Non Detections: Censored Data

Non-detections are data that have an upper or lower limit. A number of investigators have looked as censored data. The analysis of censored data is sometimes called _survival analysis_, where the term comes from the insurance industry.

An additional indicator variable $\mathbf{D}$ is used to indicate whether the data point is censored or not on the dependent variable. If $y_i$ is detected, then $D_i=1$; if $y_i$ is censored, then $D_i=0$. A common situation occurs when a source is considered detected, when the measured flux falls above some multiple of the background noise level, say $3\sigma$. Then, the probability of detecting the measured source flux $y_i$ is $p(D_i = 1 | y_i) = 1$ if $y_i > 3\sigma$ and $p(D_i = 0 | y_i) = 1$ if $y_i < 3\sigma$. Since source detection depends on measured flux, again, some sources with intrinsic flux $\eta$ above the flux limit will have a flux limit that falls below the flux limit, and vice versa.

Assume that the selection is based on the independent variable, i.e., $p(\mathbf{I} | \mathbf{x},\mathbf{y}) = p(\mathbf{I} | \mathbf{x})$.

It turns out, perhaps not surprisingly, that this situation defaults to the "selection based on measured independent variables" situation. So,

$$p(\mathbf{x}_{obs}, \mathbf{y}_{obs}, \mathbf{D} | \boldsymbol{\theta}, \boldsymbol{\psi}_{obs}) \propto \prod_{i\in \mathcal{A}_{det}} p(x_i,y_i | \boldsymbol{\theta},\psi_{obs}) \times \prod_{j \in \mathcal{A}_{cens}}
p(x_j | \psi_{obs}) \int p(D_j = 0 | y_j,x_j) p(y_i | x_j, \boldsymbol{\theta}, \boldsymbol{\psi}_{obs}) d y_i,$$

where the first product is over the set of detections, and the second product is over teh set of nondetection. 

It is noted that if the data points are measured without error, and one assumes the normal regression model $p(\eta | \xi, \boldsymbol{\theta}) = \mathcal{N} (\eta | \alpha + \beta \xi, \sigma^2)$, then the equation above becomes the censored data likelihood based on Isobe et al. 1986.

### Computational Methods

This section of the paper is devoted to a description of a Bayesian method to compute the estimates of the regression parameters $\boldsymbol{\theta}$ and their uncertainties. The way it works is that it computes the posterior probability distribution of the model parameters, given the observed data. The posterior distribution follows from Bayes' rule:

$$p(\boldsymbol{\theta}, \boldsymbol{\psi} | \mathbf{x},\mathbf{y}) \propto p(\boldsymbol{\theta},\boldsymbol{\psi}) p(\mathbf{x},\mathbf{y} | \boldsymbol{\theta}, \boldsymbol{\psi}),$$

where $p(\boldsymbol{\theta}, \boldsymbol{\psi})$ is the prior distribution for the regression parameters. Markov chain methods are described for drawing random variables from the posterior, which can be used to estimate standard errors and confidence intervals on $\boldsymbol{\theta}$ and $\boldsymbol{\psi}$.

This section is very technical; the method describing the application of the Gibbs sampler includes 13 steps. The interested student can look at this.

Kelly (2007) notes also one can get an estimate for the correlation coefficient from this using the results of the Gibbs sample, and the equation for $\hat{\rho}$ above.



###  Example:  

linmix was originally written in IDL, but has been ported to python ([see this link](https://github.com/jmeyers314/linmix)).

They helpfully reproduce the example in Kelly (2007) ([see this link](http://linmix.readthedocs.io/en/latest/example.html)), so we work through that here to see what is going on.  Monday we will see an example from my group's research.

The first simulation he performs is one without nondetections. He created $2.7 \times 10^{5}$ data sets of the independent variable from a distribution of the form:

$$p(\xi) \propto e^{\xi} (1+e^{2.75\xi})^{-1},$$

where the distribution is shown in Figure 2, along with the best-fitting one and two gaussian approximation. The two-Gaussian mixture is nearly indistinguishable from the actual distribution of $\xi$ and so should provide a good appromation to $p(\xi)$. He varied the number of points in the data sets as $n=25$, $n=50$ and $n=100$. He then simulated values of $\eta$ with $\alpha = 1.0$ and $\beta = 0.5$, using an intrinsic scatter $\epsilon$ which has a normal distribution with zero mean and $\sigma = 0.75$, with a correlation between $\eta$ and $\xi$ was $\rho = 0.62$. The joint distribution for one simulated data set is shown in Figure 3.


They simulate draws from this distribution by [rejection sampling](https://en.wikipedia.org/wiki/Rejection_sampling), a method which starts out with a random sampling, but then throws away points that are not distributed as required.  (This looks like a handy technique.)

In [ ]:
import linmix
np.random.seed(2)

In [ ]:
def pxi(xi):
    return np.exp(xi) * (1.0 + np.exp(2.75*xi))**(-1)


#fig = plt.figure()
#ax = fig.add_subplot(111)
plt.xlabel(r"$\xi$",fontsize=24)
plt.ylabel(r"$P(\xi)$",fontsize=24)

x= np.arange(-5,5, 0.01)
plt.plot(x,pxi(x))



Here is how the rejection sampling goes.  "The maximum density is a little below 0.55, so we can use that. To draw samples from $Pr(ξ)$, we first propose a value $ξ_i$ uniformly between -10 and +10 (it’s okay if we clip the tails for this example), and then keep that proposal if $Pr(ξi)>u$ where $u$ is drawn uniformly between 0 and 0.55. If $Pr(ξ_i)<u$, then we propose a new value for $ξ_i$. Here’s code to draw 100 samples from Pr(ξ):"

In [ ]:
def rejection_sample(p, pmax, prop, size):
    out=[]
    for s in range(size):
        x = prop()
        px = p(x)
        pu = np.random.uniform(low=0.0, high=pmax)
        while px < pu:
            x = prop()
            px = p(x)
            pu = np.random.uniform(low=0.0, high=pmax)
        out.append(x)
    return np.array(out)
pmax = 0.55 # max p(xi) determined by eye
prop = lambda : np.random.uniform(low=-10, high=10) # truncating range to (-10, 10)
xi = rejection_sample(pxi, pmax, prop, size=100)

Plot a histogram of the result for a sanity check.

In [ ]:
plt.xlabel(r"$\xi$",fontsize=24)
plt.ylabel(r"$P(\xi)$",fontsize=24)

x= np.arange(-5,5, 0.01)
plt.plot(x,pxi(x))
plt.hist(xi,normed=True)


From these, the measured values for $\xi$ and $\eta$ were simulated using the equations at the start of this section. The measurement errors had zero mean normal distribution of varying dispersion, and they were independent for $x$ and $y$. The variances in the measurement errors were different for each data point, and they were drawn from a scaled inverse $\chi^2$ distribution. (According to Wikipedia, the scaled inverse $\chi^2$ distribution serves as a conjugate prior for the variance parameter of a normal distribution. Clearly there is more reading to be done on this point.) The scale parameters of the inverse $\chi^2$ distribution dictate the typical size of the error boars. The parameters are denoted $t$ and $s$, and they were scaled so that $R_x \sim 0.2$, 0.5, and 0.8, while $R_y \sim 0.15$, 0.4, and 0.6. A large number of data sets were obtained.

For each simulated data set, he computed the MLE, found by maximizing this equation (from above)

$$p(\mathbf{x},\mathbf{y} | \boldsymbol{\theta}, \boldsymbol{\psi}) = \prod_{i=1}^n \sum_{k=1}^K \frac{\pi_k}{2\pi |\mathbf{V}_{k,i} | ^{1/2}} \exp \left [ -\frac{1}{2} (\mathbf{z} - \boldsymbol{\zeta}_k)^T \mathbf{V}^{-1}_{k,i} (\mathbf{z}_i - \boldsymbol{\zeta}_k) \right ].$$

He uses a $K=1$ Gaussian function. He also computes the OLS, BCES($Y|X$), and FITEXY estimates for comparison. Details about this are given in the paper. Figures 4 and 5 show the results. Note that these are distributions from many data sets, constructed to show the biases. Their use for an individual data set that you might have is to show where one or another of these methods may yield biased results. Examination of Figures 4 and 5 show that the MLE method works the best. Particularly worriesome is the OLS, which indicates slopes that aren't consistent at all with the simulation slope.

Figure 6 shows uncertainties in derived regression parameters derived by simulating draws from the posterior distribution $p(\boldsymbol{\theta}, \boldsymbol{\psi} | x,y)$. For this figure, a $K=2$ Gaussian mixture model was used. The peaks of the distributions don't line up with the input values; however, the uncertainties encompase the input values indicating that derived error range would be trustworthy. These results don't look that great, but he notes that there is dependence on the number of points, etc.

Now we will choose the regression parameters (intercept, slope, error).

In [ ]:
alpha = 1.0  # intercept
beta = 0.5  # slope
sigsqr = 0.75**2 # additional noise
epsilon = np.random.normal(loc=0, scale=np.sqrt(sigsqr), size=len(xi))
eta = alpha + beta*xi + epsilon

These are further perturbed in both the X and Y directions, and X and Y errors are assigned.  The errors have a $\chi^2$ distribution.

In [ ]:
tau = np.std(xi)
sigma = np.sqrt(sigsqr)
t = 0.4 * tau
s = 0.5 * sigma
xsig = 5*t**2 / np.random.chisquare(5, size=len(xi))
ysig = 5*s**2 / np.random.chisquare(5, size=len(eta))
x = np.random.normal(loc=xi, scale=xsig)
y = np.random.normal(loc=eta, scale=ysig)

Now plot.  The blue points show ($\xi,\eta$) the points with the intrinsic dependence as well as intrinsic scatter, whose parameters will be derived by linmix.  The green points show those points with measurement error as well in both variables.

In [ ]:
plt.axis([-10,10,-5,5],fontsize=20,linewidth=4)


plt.errorbar(xi, eta, fmt='o')

plt.errorbar(x, y, yerr=ysig, xerr=xsig,fmt='o')

Now run linmix on the simulated data.

In [ ]:
lm = linmix.LinMix(x, y, xsig, ysig, K=2)
lm.run_mcmc(silent=True)


The example notes:
"We set K=2 here to use two components in the mixture model, which is reasonable for our fairly simple (and nearly Gaussian) latent independent variable distribution.

The code will run somewhere between 5000 and 100000 steps of a MCMC to produce samples from the posterior distribution of the model parameters, given the data. The code will automatically compare the variance of sample parameters between chains to the variance within single chains to determine if convergence has been reached and stop. If you want to see status updates as the code runs, then set silent=False or just leave the silent keyword out completely (its default is False)."

The output will be in lm.chain.  Note that the burnin has been trimmed off (but it might not hurt to check that everything is copacetic!

From reading the documention on linmix [here](http://linmix.readthedocs.io/en/latest/src/linmix.html), there is a whole lot of stuff in the lm.chain.  

- alpha(float): The regression intercept.
- beta(float): The regression slope.
- sigsqr(float): The regression intrinsic scatter.
- pi(array_like): The mixture model component fractions.
- mu(array_like): The mixture model component means.
- tausqr(array_like): The mixture model component variances.
- mu0(float): The hyperparameter describing the prior variance of the distribution of mixture means.
- usqr(float): The hyperparameter describing the prior variance of the distribution of mixture variances.
- wsqr(float): The hyperparameter describing the typical scale for the prior on usqr and tausqr.
- ximean(float): The mean of the distribution for the independent latent variable xi.
- xisig(float): The standard deviation of the distribution for the independent latent variable xi.
- corr(float): The linear correlation coefficient between the latent dependent and independent variables xi and eta.



In [ ]:
print lm.chain.shape
print lm.chain[0]

In [ ]:
alpha=lm.chain[['alpha']]
beta=lm.chain[['beta']]

plt.subplot(1,2,1)
plt.ylabel('alpha',fontsize=20)
plt.plot(lm.chain[['alpha']])
plt.subplot(1,2,2)
plt.ylabel('beta',fontsize=20)
plt.plot(lm.chain[['beta']])


In [ ]:
print lm.chain['alpha'].mean(),lm.chain['alpha'].std()
print lm.chain['beta'].mean(),lm.chain['beta'].std()


plt.subplot(1,2,1)
plt.ylabel('alpha',fontsize=20)
plt.hist(lm.chain['alpha'],bins=20,histtype='step',normed=True)
plt.subplot(1,2,2)
plt.ylabel('beta',fontsize=20)
plt.hist(lm.chain['beta'],bins=20,histtype='step',normed=True)


Thus, the results are 10000 individual pairs of intercept and slope.  We can use get the best fitting line (call that the median, or you could get the MAP), and 95% confidence intervals.

In [ ]:
xplot=-5.0+0.01*np.arange(1001)
print xplot.shape

percent_regions=[1.0-0.9973,1.0-0.9545,1.0-0.6827,0.5,0.6827,0.9545,0.9973]

confidence_regions=np.zeros([7,1001])


numsamples=len(lm.chain)
temp2=np.zeros(numsamples)
f2=np.array(range(numsamples))/float(numsamples)

for i in range(1001):
    for j in range(10000):
        temp2[j]=lm.chain[j]['alpha']+xplot[i]*lm.chain[j]['beta']
    x2=np.sort(temp2)
    confidence_regions[0:7,i]=np.interp(percent_regions,f2,x2)



In [ ]:
plt.axis([-10,10,-5,5],fontsize=20,linewidth=4)


plt.errorbar(xi, eta, fmt='o')

plt.errorbar(x, y, yerr=ysig, xerr=xsig,fmt='o')
plt.plot(xplot,confidence_regions[3,:],linewidth=4)
plt.plot(xplot,confidence_regions[1,:],linewidth=4)
plt.plot(xplot,confidence_regions[5,:],linewidth=4)

### Simulations with Nondetections

He creates simulations with the same method as above, but considers sources to be "detected" only if $y$ exceeds a particular value. The results are given in Figure 8. 

Note that this choice of detection cutoff is just for convenience, e.g,. it wouldn't apply if your observations had different exposure times.  But you can still analyze such data; you just include the appropriate indicator flag.


The intrinsic distribution is shown in the left 
panel, and it is indicated which ones are detected and which ones aren't. The right panel shows the resulting MLE, plus the approximate 95% pointwise confidence intervals on the regression line. The posterior distributions are seen in Figure 9. These look pretty good.

This experiment indicates that, despite the heavy censorship, the fact that there is a relationship can be extracted.
However, note that while Kelly (2007) used an upper limit of 1.5, I softened that to 1.0, as I got a much worse result that they did.  This may be due to variance in the parameters; the particular manefistation I got (randomly) was less constraining than the one shown on the webpage.



In [ ]:
# Note that delta serves as the indicator variable.

delta = y > 1.0
notdelta = np.logical_not(delta)
ycens = y.copy()
ycens[notdelta] = 1.0
lmcens  = linmix.LinMix(x, ycens, xsig, ysig, delta=delta, K=2)
lmcens.run_mcmc(silent=True)

In [ ]:
confidence_regions_cens=np.zeros([7,1001])

numsamples=len(lmcens.chain)
temp2=np.zeros(numsamples)
f2=np.array(range(numsamples))/float(numsamples)

for i in range(1001):
    for j in range(10000):
        temp2[j]=lmcens.chain[j]['alpha']+xplot[i]*lmcens.chain[j]['beta']
    x2=np.sort(temp2)
    confidence_regions_cens[0:7,i]=np.interp(percent_regions,f2,x2)
    



In [ ]:
plt.axis([-10,10,-5,5],fontsize=20,linewidth=4)

plt.errorbar(xi, eta, fmt='o')

plt.errorbar(x[delta], y[delta], yerr=ysig[delta], xerr=xsig[delta],fmt='o')
plt.errorbar(x[notdelta], ycens[notdelta], yerr=0.3, uplims=np.ones(sum(notdelta), dtype=bool), ls=' ', c='b', alpha=0.4)
plt.plot(xplot,confidence_regions_cens[3,:],linewidth=4)
plt.plot(xplot,confidence_regions_cens[1,:],linewidth=4)
plt.plot(xplot,confidence_regions_cens[5,:],linewidth=4)

In [ ]:
print lmcens.chain['alpha'].mean(),lmcens.chain['alpha'].std()
print lmcens.chain['beta'].mean(),lmcens.chain['beta'].std()

plt.subplot(1,2,1)
plt.ylabel('alpha',fontsize=20)
plt.hist(lmcens.chain['alpha'],bins=20,histtype='step',normed=True)
plt.subplot(1,2,2)
plt.ylabel('beta',fontsize=20)
plt.hist(lmcens.chain['beta'],bins=20,histtype='step',normed=True)


### Application to Real Astronomical Data: Dependence of $\Gamma_X$ on $L_{bol} / L_{Edd}$ for Radio-Quiet Quasars

Here they are looking for a relationship between X-ray slope and the ratio of bolometric luminosity and Eddington luminosity that had been previously claimed. Both of these parameters have significant error bars. See Figure 10 and 11 for the results.

Another notable thing that he does in this paper is that he shows how much more reliable (i.e., closer to the true value) these end up compared with previous methods.  That is especially clearly seen in Fig 10 (right).